## PRÁCTICA CON CIFAR-10

### Transfer Learning para clasificación de imágenes


### Carga de Librerías

In [2]:
import numpy as np
import tensorflow as tf



#Importamos categorical para onehot
from tensorflow.keras.utils import to_categorical


#Importamos CIFAR-10
from tensorflow.keras.datasets import cifar10 #por un lado importamos

from sklearn.model_selection import train_test_split #dividir datos antes del entrenamiento


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.optimizers import Adam

from keras.src.callbacks import EarlyStopping, ReduceLROnPlateau #para preparar el entrenamiento


El dataset CIFAR-10 contiene 60,000 imágenes de 32x32 píxeles distribuidas en
10 clases (por ejemplo, aviones, automóviles, pájaros, gatos, etc.)

Este dataset
es adecuado para tareas de clasificación de imágenes y es ampliamente utilizado
en la investigación de visión por computadora

### Carga del dataset CIFAR-10

Carga del dataset: Utiliza la función load_data() para cargar el dataset
CIFAR-10 (la función está en tensorflow.keras.datasets.cifar10)

In [3]:
#Cargamos el dataset cifar-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data() #cargamos con load

#RECORDATORIO (x representa datos entrada, y etiquetas)
#x_train imagenes entrenamiento, y_train etiquetas entrenamiento
#x_test imagenes de prueba, y_test etiquetas de prueba

#Verificar las dimensiones
print("Forma de x_train:", x_train.shape)  #(50000, 32, 32, 3)
print("Forma de y_train:", y_train.shape)  #(50000, 1)
print("Forma de x_test:", x_test.shape)  #(10000, 32, 32, 3)
print("Forma de y_test:", y_test.shape)  #(10000, 1)

Forma de x_train: (50000, 32, 32, 3)
Forma de y_train: (50000, 1)
Forma de x_test: (10000, 32, 32, 3)
Forma de y_test: (10000, 1)


In [4]:
#Identificar los únicos con el train labels
classes = np.unique(y_train)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Total number of outputs :  10
Output classes :  [0 1 2 3 4 5 6 7 8 9]


Preprocesamiento de Datos: normaliza las imágenes (escalar los
valores de píxeles entre 0 y 1) y convierte las etiquetas a formato one-hot

In [5]:
#Normalizamos
x_train = x_train.astype('float32') / 255.0
y_test = y_test.astype('float32') / 255.0

#Convertimos a OneHot
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

#RECORDATORIO
#OneHot transforma etiquetas a un mejor formato para redes neuronales
#EJEMPLO:
#Antes: y = 3 → [3]
#Después: y = 3 → [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

#Nueva forma
print("Nueva forma de y_train:", y_train.shape)  # (50000, 10)
print("Nueva forma de y_test:", y_test.shape)  # (10000, 10)

Nueva forma de y_train: (50000, 10)
Nueva forma de y_test: (10000, 10)


In [6]:
#Identificar los únicos con el train labels DESPUÉS DE ONE HOT
classes = np.unique(y_train)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Total number of outputs :  2
Output classes :  [0. 1.]


#### Dividir datos antes del entrenamiento

In [7]:

#División de los datos 20 % para test
x_train, X_val, label_train ,label_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

#Comprobamos
x_train.shape, X_val.shape, label_train.shape, label_val.shape

((40000, 32, 32, 3), (10000, 32, 32, 3), (40000, 10), (10000, 10))

### Parte 2: Implementación y entrenamiento

1. Arquitecturas: utilizaremos VGG16, ResNet50 e InceptionV3

2. Preparación de las arquitecturas: carga cada uno de los modelos
preentrenados sin las capas superiores (sin la capa de clasificación) y
congela todas las capas

3. Construcción:
o VGG16: Usa VGG16 preentrenado en imagenes de ImageNet.
o ResNet50: Usa ResNet50 preentrenado.
o InceptionV3: Usa InceptionV3 preentrenado

4. Compilación: Utiliza el optimizador Adam y la función de pérdida
categorical_crossentropy

5. Entrenamiento: entrena cada modelo con las imágenes del conjunto de
entrenamiento y evalúa en el conjunto de test. Ajusta el número de épocas
según sea necesario


##### VGG16

In [8]:
#Cargar VGG16 sin la capa de clasificación final
base_model_vgg16 = VGG16(weights='imagenet', include_top=False,
input_shape=(32,32, 3))
#Congelar las capas convolucionales
for layer in base_model_vgg16.layers:
 layer.trainable = False
#Agregar capas densas personalizadas
x = Flatten()(base_model_vgg16.output)
x = Dense(256, activation='relu')(x)
x = Dense(10, activation='softmax')(x) # Para clasificación binaria
#x = Dense(1, activation='sigmoid')(x) #Para clasificación binaria
#2.4 Crear el modelo final y compilar
model_vgg16 = Model(inputs=base_model_vgg16.input, outputs=x)


##### ResNet50

In [9]:
#Cargar Resnet50 sin la capa de clasificación final
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False,
input_shape=(32, 32, 3)) #cambiamos 224,224,3
#Congelar las capas convolucionales
for layer in base_model_resnet50.layers:
 layer.trainable = False
#Agregar capas densas personalizadas
x = Flatten()(base_model_resnet50.output)
x = Dense(256, activation='relu')(x)
x = Dense(10, activation='softmax')(x) # Para clasificación binaria
#2.4 Crear el modelo final y compilar
model_resnet50 = Model(inputs=base_model_resnet50.input, outputs=x)


##### InceptionV3

In [10]:
IMG_SIZE = (75, 75, 3)  #Cambia a (32, 32, 3) si realmente necesitas ese tamaño


#Cargar InceptionV3 sin la capa de clasificación final
#base_model_inceptionv3 = ResNet50(weights='imagenet', include_top=False,
base_model_inceptionv3 = InceptionV3(weights='imagenet', include_top=False, input_shape=IMG_SIZE)
#Congelar las capas convolucionales
for layer in base_model_inceptionv3.layers:
 layer.trainable = False
#Agregar capas densas personalizadas
x = Flatten()(base_model_inceptionv3.output)
x = Dense(256, activation='relu')(x)
output_layer = Dense(10, activation='softmax')(x)  # Cambia a 1 y 'sigmoid' si es binario
#x = Dense(1, activation='sigmoid')(x) #Para clasificación binaria
#2.4 Crear el modelo final y compilar
#model_inceptionv3 = Model(inputs=base_model_inceptionv3.input, outputs=x)
model_InceptionV3 = Model(inputs=base_model_inceptionv3.input, outputs=output_layer)



In [11]:

#Descongelar algunas capas superiores para ajustar mejor
for layer in model_vgg16.layers[-4:]: # Descongelar 4 últimas capas
 layer.trainable = True
 

model_vgg16.compile(optimizer=Adam(learning_rate=0.0001),
loss='binary_crossentropy', metrics=['accuracy'])
model_vgg16.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,586 (56.64 MB)

 Trainable params: 133,898 (523.04 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [12]:
#Descongelar algunas capas superiores para ajustar mejor
for layer in model_resnet50.layers[-4:]: #Descongelar 4 últimas capas
 layer.trainable = True

model_resnet50.compile(optimizer=Adam(learning_rate=0.0001),
loss='binary_crossentropy', metrics=['accuracy'])
model_resnet50.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 38, 38, 3) │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 16, 16,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 16, 16,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 16, 16,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 18, 18,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 8, 8, 64)  │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 8, 8, 64)  │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 8, 8, 64)  │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 8, 8, 64)  │          0 │ conv2_block1_1_b… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 8, 8, 64)  │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 8, 8, 64)  │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 8, 8, 64)  │          0 │ conv2_block1_2_b… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 8, 8, 256) │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 8, 8, 256) │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 8, 8, 256) │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 8, 8, 256) │      1,024 │ conv2_block1_3_c

 Total params: 24,114,826 (91.99 MB)

 Trainable params: 527,114 (2.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [13]:
#Descongelar algunas capas superiores para ajustar mejor
for layer in model_InceptionV3.layers[-4:]: #Descongelar 4 últimas capas
 layer.trainable = True

model_InceptionV3.compile(optimizer=Adam(learning_rate=0.0001),
loss='binary_crossentropy', metrics=['accuracy'])
model_InceptionV3.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 75, 75, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 37, 37,    │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 37, 37,    │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 37, 37,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 35, 35,    │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 35, 35,    │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 35, 35,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 35, 35,    │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 35, 35,    │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 35, 35,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 17, 17,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 17, 17,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 17, 17,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 17, 17,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 15, 15,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 15, 15,    │          0 │ batch_normalizat

 Total params: 22,329,898 (85.18 MB)

 Trainable params: 527,114 (2.01 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

#### Entrenamiento con VGG16

In [14]:

#Le ponemos early stopping y reduce lr
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

#Entrenamiento y almacenamiento de los historiales en array
histories = {}

batch_size = 32
epochs = 20
num_classes = 10 #número de clases
histories['VGG16'] = model_vgg16.fit(
    x_train, label_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_val, label_val),
    callbacks=[early_stopping, reduce_lr]
)



Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 80s 63ms/step - accuracy: 0.3033 - loss: 0.3395 - val_accuracy: 0.4893 - val_loss: 0.2352 - learning_rate: 1.0000e-04
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 81s 65ms/step - accuracy: 0.5110 - loss: 0.2292 - val_accuracy: 0.5240 - val_loss: 0.2199 - learning_rate: 1.0000e-04
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 78s 62ms/step - accuracy: 0.5417 - loss: 0.2159 - val_accuracy: 0.5414 - val_loss: 0.2123 - learning_rate: 1.0000e-04
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 76s 61ms/step - accuracy: 0.5633 - loss: 0.2082 - val_accuracy: 0.5546 - val_loss: 0.2076 - learning_rate: 1.0000e-04
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 78s 62ms/step - accuracy: 0.5721 - loss: 0.2036 - val_accuracy: 0.5669 - val_loss: 0.2038 - learning_rate: 1.0000e-04
Epoch 6/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 80s 64ms/step - accuracy: 0.5782 - loss: 0.2001 - val_accuracy: 0.5719 - val_loss: 0.2015 - learning_rate: 1.0000e-04
Epoch 7/20
1250/1250 ━━━━━━━━━━━━━━━━━━━

In [ ]:

#Le ponemos early stopping y reduce lr
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

#Entrenamiento y almacenamiento de los historiales en array
histories = {}

batch_size = 38
epochs = 50
num_classes = 10 #número de clases
histories['VGG16'] = model_vgg16.fit(
    x_train, label_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_val, label_val),
    callbacks=[early_stopping, reduce_lr]
)



Epoch 1/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 77s 72ms/step - accuracy: 0.6342 - loss: 0.1746 - val_accuracy: 0.6022 - val_loss: 0.1855 - learning_rate: 1.0000e-04
Epoch 2/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 79s 75ms/step - accuracy: 0.6378 - loss: 0.1740 - val_accuracy: 0.6054 - val_loss: 0.1850 - learning_rate: 1.0000e-04
Epoch 3/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 77s 73ms/step - accuracy: 0.6397 - loss: 0.1731 - val_accuracy: 0.6068 - val_loss: 0.1845 - learning_rate: 1.0000e-04
Epoch 4/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 78s 74ms/step - accuracy: 0.6445 - loss: 0.1712 - val_accuracy: 0.6042 - val_loss: 0.1840 - learning_rate: 1.0000e-04
Epoch 5/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 80s 76ms/step - accuracy: 0.6501 - loss: 0.1696 - val_accuracy: 0.6100 - val_loss: 0.1838 - learning_rate: 1.0000e-04
Epoch 6/50
1053/1053 ━━━━━━━━━━━━━━━━━━━━ 79s 75ms/step - accuracy: 0.6459 - loss: 0.1699 - val_accuracy: 0.6071 - val_loss: 0.1839 - learning_rate: 1.0000e-04
Epoch 7/50
1053/1053 ━━━━━━━━━━━━━━━━━━━

#### ENTRENAMIENTO CON RESNET 50

In [ ]:

#Le ponemos early stopping y reduce lr
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

#Entrenamiento y almacenamiento de los historiales en array
histories = {}

batch_size = 32
epochs = 20
num_classes = 10 #número de clases
histories['N'] = model_resnet50.fit(
    x_train, label_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_val, label_val),
    callbacks=[early_stopping, reduce_lr]
)



#### Entrenamiento con InceptionV3

### Parte 3: Evaluación

##### Métricas

##### 2.5 Entrenamiento

VGG16

In [ ]:
#Le ponemos early stopping y reduce lr
early_stopping = EarlyStopping(monitor='val loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

#Entrenamiento y almacenamiento de los historiales en array
histories = {}

batch_size = 32
epochs = 20
num_classes = 10 #número de clases
histories['VGG16'] = model_vgg16.fit(
    x_train, label_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_val, label_val),
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/20


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(32, 10), output.shape=(32, 1)

ResNet50

Epoch 1/20


ValueError: Input 0 of layer "functional_11" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(32, 32, 32, 3)

In [ ]:
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
#Carga de VGG16 con pesos de ImageNet
vgg_model = VGG16(weights='imagenet')
resnet_model = ResNet50()
inception_model = InceptionV3()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

# Cargar VGG16 sin la capa de clasificación final
base_model = VGG16(weights='imagenet', include_top=False,
input_shape=(224, 224, 3))

# Congelar las capas convolucionales
for layer in base_model.layers:
 layer.trainable = False
 
# Agregar capas densas personalizadas
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x) # Para clasificación binaria

# Crear el modelo final y compilar
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(learning_rate=0.0001),
loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_25 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,137,729 (80.63 MB)

 Trainable params: 6,423,041 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
#Redimensionamos a 75
x_train_resized = tf.image.resize(x_train, (75,75))
x_test_resized = tf.image.resize(x_test, (75,75))

In [ ]:
#Entrenamiento del modelo InceptionV3
historia_inceptionv3 = model.fit(
    x_train_resized, y_train,
    validation_data=(x_test_resized, y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)



NameError: name 'epochs' is not defined